In [ ]:
# Load Packages #

from __future__ import print_function

import collections
import csv
import logging

import SimpleITK as sitk

import radiomics
from radiomics import featureextractor
import pydicom
import pylab
from pydicom import dcmread

from numpy import asarray
import numpy as np
import pandas as pd
import nrrd

import os
from os import path, rename
from PIL import Image
import PIL.ImageOps

In [ ]:
Output = '/home/h63914/Documents/Pyradiomics_Input_Output/Output/FullEcho/PW/'

for filename in os.listdir('/home/h63914/Documents/Pyradiomics_Input_Output/Input/FullEcho/PW/'):
    if filename.endswith('FullEcho_segmentation_PW.png'):
        ##Open Mask File##
        mask = Image.open(os.path.join('/home/h63914/Documents/Pyradiomics_Input_Output/Input/FullEcho/PW/', filename))
        ##Convert to B&W##
        thresh = 100
        fn = lambda x : 255 if x > thresh else 0
        mask1 = mask.convert('L').point(fn, mode='L')
#        mask1.save(filename)
        ##Prepare for Cropping##
        width, height = mask1.size   # Get dimensions
        left = width/4.47
        top = height/4.50
        right = 3 * width/3.75
        bottom = 3 * height/3.38
        mask1_cropped = mask1.crop((left, top, right, bottom))
        mask1_cropped.save(path.join(Output, filename))
    if filename.endswith('FullEcho_original_PW.png'):
        ##Open Image File##
        image1 = Image.open(os.path.join('/home/h63914/Documents/Pyradiomics_Input_Output/Input/FullEcho/PW/', filename))
        ##Prepare for Cropping##
        width, height = image1.size   # Get dimensions
        left = width/4.47
        top = height/4.50
        right = 3 * width/3.75
        bottom = 3 * height/3.38
        image1_cropped = image1.crop((left, top, right, bottom))
        image1_cropped.save(path.join(Output, filename))

In [ ]:
def main():
  outPath = "/home/h63914/Documents/Pyradiomics_Input_Output/Input/"

  inputCSV = os.path.join(outPath, 'File_Location_PW_FullEcho.csv')
  outputFilepath = os.path.join(outPath, 'Radiomics_Features_PW_FullEcho.csv')
  progress_filename = os.path.join(outPath, 'Pyradiomics_Log_PW_FullEcho.txt')
  params = os.path.join(outPath, './', 'Params_FINAL_LV.yaml')

  # Configure logging
  rLogger = logging.getLogger('radiomics')

  # Create handler for writing to log file
  handler = logging.FileHandler(filename=progress_filename, mode='w')
  handler.setFormatter(logging.Formatter('%(levelname)s:%(name)s: %(message)s'))
  rLogger.addHandler(handler)

  # Initialize logging for batch log messages
  logger = rLogger.getChild('batch')

  # Set verbosity level for output to stderr (default level = WARNING)
  radiomics.setVerbosity(logging.INFO)

  logger.info('pyradiomics version: %s', radiomics.__version__)
  logger.info('Loading CSV')

  flists = []
  try:
    with open(inputCSV, 'r') as inFile:
      cr = csv.DictReader(inFile, lineterminator='\n')
      flists = [row for row in cr]
  except Exception:
    logger.error('CSV READ FAILED', exc_info=True)

  logger.info('Loading Done')
  logger.info('Patients: %d', len(flists))

  if os.path.isfile(params):
    extractor = featureextractor.RadiomicsFeatureExtractor(params)
  else:  # Parameter file not found, use hardcoded settings instead
    settings = {}
    settings['binWidth'] = 25
    settings['resampledPixelSpacing'] = None  # [3,3,3]
    settings['interpolator'] = sitk.sitkBSpline
    settings['enableCExtensions'] = True

    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
    # extractor.enableInputImages(wavelet= {'level': 2})

  logger.info('Enabled input images types: %s', extractor.enabledImagetypes)
  logger.info('Enabled features: %s', extractor.enabledFeatures)
  logger.info('Current settings: %s', extractor.settings)

  headers = None

  for idx, entry in enumerate(flists, start=1):

    logger.info("(%d/%d) Processing Patient (Image: %s, Mask: %s)", idx, len(flists), entry['Image'], entry['Mask'])

    imageFilepath = entry['Image']
    maskFilepath = entry['Mask']
    label = entry['Label']

    if (imageFilepath is not None) and (maskFilepath is not None):
      featureVector = collections.OrderedDict(entry)
      featureVector['Image'] = os.path.basename(imageFilepath)
      featureVector['Mask'] = os.path.basename(maskFilepath)

      try:
        featureVector.update(extractor.execute(sitk.ReadImage(imageFilepath,sitk.sitkUInt8), sitk.ReadImage(maskFilepath,sitk.sitkUInt8), label=255))

        with open(outputFilepath, 'a') as outputFile:
          writer = csv.writer(outputFile, lineterminator='\n')
          if headers is None:
            headers = list(featureVector.keys())
            writer.writerow(headers)

          row = []
          for h in headers:
            row.append(featureVector.get(h, "N/A"))
          writer.writerow(row)
      except Exception:
        logger.error('FEATURE EXTRACTION FAILED', exc_info=True)

if __name__ == '__main__':
  main()